In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
from PIL import Image

import cv2
import os
import numpy as np

from dataset_MH import *
from googlenet import *

print("torch-version:", torch.__version__)
print("Available GPU:", torch.cuda.current_device())

torch-version: 1.1.0
Available GPU: 0


In [2]:
# Params
input_size = (1200, 800) #1200x800
batch_size = 12 #12
num_workers = 12 #12

# 기존 데이터 경로
NORMAL_DIR = '../snapshots_pureok'
FAULT_DIR = '../snapshots'

# 추가 데이터 경로
ADD_NORMAL_DIR = '../snapshots_punch181'
ADD_FAULT_DIR = '../snapshots_punch181_NG'

# 카메라 번호 (1~4까지 있음, 카메라별로 각각 다 따로 학습해야됩니다.) CAM1 / CAM2 / CAM3 / CAM4
cam1 = 'CAM1'
cam2 = 'CAM2'
cam3 = 'CAM3'
cam4 = 'CAM4'

In [3]:
# 기존 데이터 로드
normal_list1 = filename_list(NORMAL_DIR, cam=cam1)
fault_list1 = filename_list(FAULT_DIR, cam=cam1)
normal_list2 = filename_list(NORMAL_DIR, cam=cam2)
fault_list2 = filename_list(FAULT_DIR, cam=cam2)
normal_list3 = filename_list(NORMAL_DIR, cam=cam3)
fault_list3 = filename_list(FAULT_DIR, cam=cam3)
normal_list4 = filename_list(NORMAL_DIR, cam=cam4)
fault_list4 = filename_list(FAULT_DIR, cam=cam4)

mhdb9 = MyungHwa_ansan_db(normal_list1, fault_list1, dataType='test', input_size=input_size, augmentation=True)
mhdb10 = MyungHwa_ansan_db(normal_list2, fault_list2, dataType='test', input_size=input_size, augmentation=True)
mhdb11 = MyungHwa_ansan_db(normal_list3, fault_list3, dataType='test', input_size=input_size, augmentation=True)
mhdb12 = MyungHwa_ansan_db(normal_list4, fault_list4, dataType='test', input_size=input_size, augmentation=True)
   
# 추가 데이터 로드
add_normal_list1 = filename_list(ADD_NORMAL_DIR, cam=cam1)
add_fault_list1 = filename_list(ADD_FAULT_DIR, cam=cam1)
add_normal_list2 = filename_list(ADD_NORMAL_DIR, cam=cam2)
add_fault_list2 = filename_list(ADD_FAULT_DIR, cam=cam2)
add_normal_list3 = filename_list(ADD_NORMAL_DIR, cam=cam3)
add_fault_list3 = filename_list(ADD_FAULT_DIR, cam=cam3)
add_normal_list4 = filename_list(ADD_NORMAL_DIR, cam=cam4)
add_fault_list4 = filename_list(ADD_FAULT_DIR, cam=cam4)

add_mhdb9 = MyungHwa_ansan_db(add_normal_list1, add_fault_list1, dataType='test', input_size=input_size, augmentation=True)
add_mhdb10 = MyungHwa_ansan_db(add_normal_list2, add_fault_list2, dataType='test', input_size=input_size, augmentation=True)
add_mhdb11 = MyungHwa_ansan_db(add_normal_list3, add_fault_list3, dataType='test', input_size=input_size, augmentation=True)
add_mhdb12 = MyungHwa_ansan_db(add_normal_list4, add_fault_list4, dataType='test', input_size=input_size, augmentation=True)
    
# 데이터셋 합치기
concat_mhdb9 = torch.utils.data.ConcatDataset([mhdb9, add_mhdb9])
concat_mhdb10 = torch.utils.data.ConcatDataset([mhdb10, add_mhdb10])
concat_mhdb11 = torch.utils.data.ConcatDataset([mhdb11, add_mhdb11])
concat_mhdb12 = torch.utils.data.ConcatDataset([mhdb12, add_mhdb12])

test_mhdb_batch1 = torch.utils.data.DataLoader(concat_mhdb9, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_mhdb_batch2 = torch.utils.data.DataLoader(concat_mhdb10, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_mhdb_batch3 = torch.utils.data.DataLoader(concat_mhdb11, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_mhdb_batch4 = torch.utils.data.DataLoader(concat_mhdb12, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [4]:
model = GoogLeNet(in_channel=3).cpu()
model.load_state_dict(torch.load('./saved_models/GoogLeNet-MH-Epoch-180-Loss-24.89314.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [5]:
#eval accuracy
correct = 0
total = 0

with torch.no_grad():
    model.eval()
    for img, label in test_mhdb_batch1:
        img = img.cpu()
        label = label.cpu()
        outputs = model(img)
        #print(outputs)
        
        value, predicted = torch.max(outputs, dim=0)
        #print("value:", value)
        #print(predicted)
        
        total += label.size(0)
        #print(total)
        correct += (predicted.float() == label.float()).sum().item()
        #print(correct)

print('Accuracy of the model on test images: %d%%' % (100 * correct / total))

Accuracy of the model on test images: 7%
